In [5]:
test_path = '/home/projects/vokquant/accent-recog-slt2022/CommonAccent/data/at_states'
source_path = '/home/projects/vokquant/accent-recog-slt2022/CommonAccent/results/ECAPA-TDNN/AT/spkrec-ecapa-voxceleb/10006/save/CKPT+2024-10-03+11-03-55+00/embedding_model.ckpt'
hparams_path = '/home/projects/vokquant/accent-recog-slt2022/CommonAccent/accent_id/hparams/train_w2v2_xlsr_at.yaml'


import pandas as pd
import numpy as np

# MMS embedding

In [2]:
from transformers import Wav2Vec2ForSequenceClassification, AutoFeatureExtractor
import torch
import soundfile as sf

# Load the model and processor once
model_id_mms = "facebook/mms-lid-4017"
processor_mms = AutoFeatureExtractor.from_pretrained(model_id_mms)
model_mms = Wav2Vec2ForSequenceClassification.from_pretrained(model_id_mms)

def get_embedding_mms(audio_file, model, processor):
    # Load audio file
    speech, sample_rate = sf.read(audio_file)

    # Process the audio using the feature extractor
    # if stereo, use only one channel
    if len(speech.shape) > 1:
        speech = speech[:, 0]
    inputs = processor(speech, sampling_rate=sample_rate, return_tensors="pt", padding=True)

    # Forward pass through the model to get both logits and hidden states
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    # Extract the logits for language identification
    logits = outputs.logits

    # Get the predicted language
    lang_id = torch.argmax(logits, dim=-1)[0].item()
    detected_lang = model.config.id2label[lang_id]

    # Extract the embeddings from the last hidden state
    embeddings = outputs.hidden_states[-1]

    # Optionally, calculate mean embeddings if you need a single vector per input
    mean_embeddings = torch.mean(embeddings, dim=1)

    # Return the detected language and embeddings
    return detected_lang, mean_embeddings

Some weights of the model checkpoint at facebook/mms-lid-4017 were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/mms-lid-4017 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametri

In [6]:
# fine-tune the model using adapter
def fine_tune_model(model, processor, hparams, source_path, test_path):
    # Load the model from the source path
    model.load_state_dict(torch.load(source_path))

    # Load the hyperparameters
    hparams = hparams

    # Load the test data
    test_data = pd.read_csv(test_path)

    # Load the optimizer and scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=hparams["learning_rate"])
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams["learning_rate"], 
                                                    steps_per_epoch=len(test_data), epochs=hparams["epochs"])

    # Fine-tune the model
    for epoch in range(hparams["epochs"]):
        for i, row in test_data.iterrows():
            # Get the embeddings
            lang, embeddings = get_embedding_mms(row["path"], model, processor)

            # Get the label
            label = torch.tensor([row["label"]])

            # Calculate the loss
            loss = torch.nn.functional.cross_entropy(embeddings, label)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            # Print the loss
            if i % 100 == 0:
                print(f"Epoch {epoch}, Step {i}, Loss: {loss.item()}")
                
    return model

# Load the model and processor
model = model_mms
processor = processor_mms

# Load the hyperparameters
hparams = torch.load(hparams_path)

# Fine-tune the model
fine_tune_model(model, processor, hparams, source_path, test_path)

# Save the model
torch.save(model.state_dict(), "fine_tuned_model.ckpt")
print("Model saved successfully!")


/tmp/ipykernel_3849465/63577844.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  hparams = torch.load(hparams_path)


UnpicklingError: invalid load key, '#'.

In [4]:
from transformers import Wav2Vec2Processor
from fairseq.models.wav2vec import Wav2Vec2Model
from adapters import AutoAdapterModel

# Load MMS LID model
processor = Wav2Vec2Processor.from_pretrained('facebook/mms-lid-4017')
mms_model = Wav2Vec2Model.from_pretrained('facebook/mms-lid-4017')
adapter_model = AutoAdapterModel.from_pretrained('facebook/mms-lid-4017')

# Add a language identification adapter
adapter_name = "deu"
adapter_model.add_adapter(adapter_name)

# Activate the adapter
adapter_model.set_active_adapters(adapter_name)


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /facebook/mms-lid-4017/resolve/main/preprocessor_config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /facebook/mms-lid-4017/resolve/main/tokenizer_config.json HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /facebook/mms-lid-4017/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /facebook/mms-lid-4017/resolve/main/tokenizer_config.json HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /facebook/mms-lid-4017/resolve/main/vocab.json HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /facebook/mms-lid-4017/resolve/main/tokenizer_config.json HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /facebook/mms-lid-4017/resolve/main/added_tokens.json HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /facebook/mms-l

OSError: Can't load tokenizer for 'facebook/mms-lid-4017'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'facebook/mms-lid-4017' is the correct path to a directory containing all relevant files for a Wav2Vec2CTCTokenizer tokenizer.